# Assignment 1

**Deadline**:  Sunday 21 November, 23.59.

Your notebook should not give errors when executed with `Run All`. Please submit your answers via [Canvas](https://canvas.uva.nl/courses/25150/assignments/248822).

| |Name |Student number|Email|
|:-|:----|:-------------|:----|
|1.|  |        |     |
|2.|  |        |     |

****Hand in the following****:
* Your notebook. N.B. **click on `Kernel`, then `Restart & Run All`** before submitting, see notes.
* A (printed) pdf version of your notebook.

****NOTES****:
* The assignment is a partial stand-in for a final examination, so the usual rules regarding plagiarism and fraud apply, with all attendant consequences. Code found on the internet or elsewhere is not acceptable as a solution.
* Before submitting your work, **click on `Kernel`, then `Restart & Run All`** and verify that your notebook produces the desired results and does not error.
* Make sure that any function you write has a docstring, and comments where appropriate.
* Some questions require you to write code to obtain a numerical result (e.g., an option price). In that case, don't just give the function, but also the result of calling it with the given parameter values (i.e., the numerial value that it returns). If your function uses random numbers, then set the seed to 0 before calling it. This makes it much easier to grade the assignments (at least as long as the answer is correct).
* How to convert your notebook to pdf: The easiest way is probably to use your browser's print functionality: e.g. in Chrome, press the three dots to go to Options, select 'Print...', and then select 'save as pdf' as Destination. I usually set it to 70% zoom such that the output fits the printed pdf pages.

**Declaration of Originality**:

We whose names are given under 1. and 2. above declare that
1. These solutions are solely our own work.
2. We have not made these solutions available to any other student.

## Question 1: Testing the CAPM
The CAPM implies that
\begin{equation*}
E(R_{i}-R_{f})=\beta _{i}E(R_{m}-R_{f}),\qquad \beta _{i}=\frac{\mathrm{cov}%
(R_{i},R_{m})}{\mathrm{var}(R_{m})},
\end{equation*}

where $R_i$ is the return on a stock, and $R_m$ is the market return. Suppose we have time series on $n$ different stock or portfolio returns $\{R_{it},i=1,\ldots ,n\}_{t=1}^{T}$ and on a "market return" (value weighted index) $\{R_{mt}\}_{t=1}^{T}$.  We also have observations on a risk-free interest rate $\{R_{ft}\}_{t=1}^{T}$ and construct with these the excess returns $r_{it}=R_{it}-R_{ft}$ and $r_{mt}=R_{mt}-R_{ft}$. Now $\beta_i$ can be estimated from the time-series regression \begin{equation*}
r_{it}=\alpha _{i}+\beta _{i}r_{mt}+\varepsilon _{it},\qquad t=1,\ldots ,T.
\end{equation*}

One way to test the CAPM is as follows: denote by $\bar{r}_{i}$
and $\hat{\beta}_{i}$ the average excess return and estimated $\beta$ of stock $i$. If the model $E(r_{i})=\beta_{i}E(r_{m})$ is valid, then $(\hat{\beta}_{i},\bar{r}_{i})$ should lie on a line with zero intercept and slope $\lambda =E(r_{mt})$. This line is called the *security market line*, and $\lambda$ is known as the *market risk premium*.

We can estimate $\lambda $ by OLS in the cross-section regression
\begin{equation*}
\bar{r}_{i}=\lambda \hat{\beta}_{i}+\alpha _{i},\qquad i=1,\ldots ,n.
\end{equation*}
Note that $\hat{\beta_i}$ is the regressor, $\lambda$ the coefficient, and $\alpha_i$ the error term.

The assignment is to estimate the CAPM betas of the 30 constituent stocks of the Dow (using the return on the Dow as the market return, and the 3 month T-bill rate as the risk-free rate), then estimate the above cross-section regression, and finally make a plot of the security market line superimposed on a scatter plot of $(\hat{\beta}_{i},\bar{r}_{i})$.

**1.1** Import the relevant libraries:

In [ ]:
import pandas as pd
import pandas_datareader.data as web
import numpy as np
import matplotlib.pyplot as plt
import time
import statsmodels.formula.api as smf
%matplotlib inline

**1.2** Obtain, from the FRED database, the daily adjusted closing prices of the Dow Jones Industrial Average (DJIA) from 31/12/2011 to 30/09/2021. Convert them into percentage log returns $r_t=100 \times \log(P_t/P_{t-1})$ and store them in a DataFrame `df_1`, which has the date as index and 'DJIA' as column name:

In [ ]:
df_1 = pd.DataFrame()
startdate = pd.datetime(2011, 12, 31)
enddate = pd.datetime(2021, 9, 30) 
df = web.DataReader('DJIA','fred', start=startdate, end=enddate)
df_1 = 100*(np.log(df) - np.log(df.shift(1)))

**1.3** Obtain, also from the FRED database, daily data on the 3-month T-bill rate (DTB3) for the same period and divide them by 365:

In [ ]:
DTB3 = pd.DataFrame()
DTB = web.DataReader('DTB3','fred', start=startdate, end=enddate)
DTB3 = DTB/365
print(DTB)

**1.4** The csv file `tickerdata.csv` which has been provided with this assignment contains the adjusted closing prices for the 30 constituent stocks of the Dow Jones.

Import the dataset into a second DataFrame `df_2`, convert the `Date`-column to datetime, and make the `Date`-column the new index:

In [ ]:
df2 = pd.read_csv('/Applications/tickerdata.csv')
df2['Date']= pd.to_datetime(df2['Date'])
df2.set_index('Date', inplace=True)
print(df2)

**1.5** Using a `for` loop (over the columns), convert all the given adjusted closing prices into percentage log returns $r_t=100\log(P_t/P_{t-1})$:

In [ ]:
for returns in df_2:
    returns = np.log(df2/df2.shift(1))
df_2 = returns


**1.6** Concatenate DataFrames `df_1` and `df_2` along the common axis to a final DataFrame `df`:


Notes (also hold for next question):
* Need to specify the axis
* `dtype` of the indices should be the same

In [ ]:
returns=pd.concat([df_1,df_2],axis=1)

**1.7** Convert the raw returns in `df` to excess returns by subtracting the `pandas` series `rf` from all columns:

In [ ]:
excess_returns = returns.sub(DTB3['DTB3'], axis = 'index')

**1.8** Drop all rows from `df` that contain at least one NaN:

In [ ]:
excess_returns = excess_returns.dropna()

**1.9** Use a `for` loop to estimate a CAPM time series regression for each stock, and store the estimated slope coefficient in a list. 

*Hint*: use string interpolation to construct the regression equation and use the list `tickers` below, containing the ticker symbols of all 30 constituent stocks of the DJIA.

In [ ]:
tickers = ["AAPL", "AMGN", "AXP", "BA", "CAT", "CRM", "CSCO", "CVX", "DIS", "DD",
            "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KO", "MCD", "MMM", 
            "MRK", "MSFT", "NKE", "PG", "TRV", "UNH", "V", "VZ", "WBA","WMT"]

In [ ]:
list1 = []
list2 = []

for i in excess_returns:
    if i != 'DJIA':
        list1.append(i)
        model = smf.ols('excess_returns[i]~ + DJIA',data=excess_returns)
        result=model.fit(cov_type='HAC',cov_kwds={'maxlags':5})
        list1.append(result.params[1])
        list2.append(list1)
        list1 = []
print(list2)
df4 = pd.DataFrame(list2, columns=['tickers', 'beta'])
df4.set_index('tickers', inplace=True)
print(df4)

**1.10** Create a new dataframe that has `tickers` as index, and two columns: `beta`, containing the 30 estimated betas, and `meanret`, containing the mean excess returns of the 30 stocks:

In [2]:
list3 = []

for i in excess_returns:
    if i != 'DJIA':
        list1.append(i)
        list1.append(excess_returns[i].mean())
        list3.append(list1)
        list1 = []
print(list3)
df5 = pd.DataFrame(list3, columns=['tickers', 'meanret'])
df5.set_index('tickers', inplace=True)

df6=pd.concat([df4,df5],axis=1)

print(df6)

NameError: name 'excess_returns' is not defined

**1.11** Estimate the security market line by a cross-sectional regression (without intercept), and print a summary of the result:

In [ ]:
model = smf.ols('beta~ -1 + meanret', data=df6)
result=model.fit(cov_type='HAC',cov_kwds={'maxlags':5})
print(result.summary2())

**1.12** Make a scatter plot of $(\hat{\beta}_{i},\bar{r}_{i})$ and overlay it with a red regression line (the security market line). Add a title and legend, and label the axes:

In [3]:
plt.plot(df6['beta'], df6['meanret'], 'o')
m, b = np.polyfit(df6['beta'],df6['meanret'], 1)
plt.plot(df6['beta'], m*df6['beta'] + b, 'r-')
plt.xlabel('beta')
plt.ylabel('mean excess returns')
plt.title('Down Jones')
plt.legend(['individual stock', 'security market line'])

NameError: name 'plt' is not defined

## Question 2: Value at Risk
In this question we construct a VaR estimates for two stocks, one high risk and one low risk stock. Further, for the high risk stock, we split the sample into two periods to evaluate the VaR out-of-sample. You can re-use the data from the previous question.

**2.1** Use the above estimates to identify the most risky and the least risky stock in terms of their estimated betas. Create two `pandas` series `r_hirisk` and `r_lorisk` from the `df` DataFrame.

**2.2** For both stocks, create a 1% VaR estimate, assuming a $t$ distribution. Comment on the difference between the estimates.

**Answer the following questions for the *risky stock* only.**

**2.3** Test the fit of the Student's *t* distribution using a QQ plot, and give a conclusion in words.

**2.4** Make a plot consisting of two subplots, the left of which shows $-R_t$ and $VaR^{0.01}$, and the right panel showing a histogram of $R_t$ overlaid with the fitted Student's *t* distribution. Make sure to add title, legend, and axis labels.

*Hint*: The Value at Risk $VaR^{0.01}$ is constant over time, so the plot of the VaR will be a horizontal line.

**2.5** Finally, split the sample of `r_hirisk` into two parts: one sample from 2012-2019, and one sample from 2020-2021. Use the first sample (2012-2019) to compute the 1% VaR assuming a Student's $t$ distribution as before. Then, use the second sample to evaluate the VaR by counting the number of exceedances in the period 2020-2021. Give a conclusion in words.

*Bonus* Does your conclusion change if you evaluate the VaR based on returns in 2021 only?